# Training

This notebook loads the ETL data and trains the LSTM. 

In [ ]:
!mamba install -y numpy==1.19

In [ ]:
# %load_ext lab_black

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import os.path

import fsspec
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
mapper = fsspec.get_mapper(f'gs://carbonplan-scratch/metml/etl/x_train.zarr')
x_train = xr.open_zarr(mapper, consolidated=True)['x'].load()

mapper = fsspec.get_mapper(f'gs://carbonplan-scratch/metml/etl/x_val.zarr')
x_val = xr.open_zarr(mapper, consolidated=True)['x'].load()

mapper = fsspec.get_mapper(f'gs://carbonplan-scratch/metml/etl/y_train.zarr')
y_train = xr.open_zarr(mapper, consolidated=True)['y'].load()

mapper = fsspec.get_mapper(f'gs://carbonplan-scratch/metml/etl/y_val.zarr')
y_val = xr.open_zarr(mapper, consolidated=True)['y'].load()
x_train

In [ ]:
xdims = dict(zip(x_train.dims, x_train.shape))
ydims = dict(zip(y_train.dims, y_train.shape))
print(xdims, ydims)

In [ ]:
input_shape = (xdims["lookback"], xdims["features"])
print(input_shape)


from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend

# root mean squared error (rmse) for regression (only for Keras tensors)
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))


# mean squared error (mse) for regression  (only for Keras tensors)
def mse(y_true, y_pred):
    return backend.mean(backend.square(y_pred - y_true), axis=-1)


# coefficient of determination (R^2) for regression  (only for Keras tensors)
def r_square(y_true, y_pred):
    SS_res = backend.sum(backend.square(y_true - y_pred))
    SS_tot = backend.sum(backend.square(y_true - backend.mean(y_true)))
    return 1 - SS_res / (SS_tot + backend.epsilon())


def bias(y_true, y_pred):
    return backend.mean(y_pred) - backend.mean(y_true)


metrics = [rmse, mse, r_square, bias]
# metrics = []

def make_model_1(var, ydims=None):
    # design network
    name = f"1_layer_lstm_{var}"
    model = Sequential(name=name)
    model.add(LSTM(20, input_shape=input_shape, use_bias=True))
    model.add(Dense(ydims["features"]))
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=metrics)
    return model


def make_model_2(var, ydims=None):
    # design network
    name = f"2_layer_lstm_{var}"
    model = Sequential(name=name)
    model.add(LSTM(20, input_shape=input_shape, use_bias=True, return_sequences=True))
    model.add(LSTM(20))
    model.add(Dense(ydims["features"]))
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=metrics)
    return model


def make_model_3(var, ydims=None):
    # design network
    name = f"3_layer_lstm_{var}"
    model = Sequential(name=name)
    model.add(LSTM(20, input_shape=input_shape, use_bias=True, return_sequences=True))
    model.add(LSTM(20, return_sequences=True))
    model.add(LSTM(20))
    model.add(Dense(ydims["features"]))
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=metrics)
    return model


def make_model_4(var, ydims=None):
    # design network
    name = f"3_layer_lstm_wide_{var}"
    model = Sequential(name=name)
    model.add(LSTM(40, input_shape=input_shape, use_bias=True, return_sequences=True))
    model.add(LSTM(40, return_sequences=True))
    model.add(LSTM(40))
    model.add(Dense(ydims["features"]))
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=metrics)
    return model

In [ ]:
# train the model
# history = {}
# for batch_size in [128, 512, 2048, 8192, 16384]:
#     model = make_model_1()
#     history[batch_size] = model.fit(x_train.values, y_train.values,
#                         validation_data=(x_val.values, y_val.values),
#                          batch_size=batch_size, epochs=30,
#                          shuffle=True, callbacks=callbacks)

# plt.figure(figsize=(12, 12))
# # plot training history
# for batch, h in history.items():
# #     plt.plot(h.history['loss'], label=f'train-{batch}')
#     plt.plot(h.history['val_loss'], label=f'test-{batch}')
# plt.yscale('log')
# plt.xscale('log')
# plt.legend()

# based on this, I'm using the batch_size of 512 for now

In [ ]:
y_val[:100].plot.line(x="samples")

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
batch_size = 512
history = {}


def make_callbacks(name):
    mc = ModelCheckpoint(
        f"best_{name}.h5",
        monitor="val_mse",
        mode="max",
        verbose=0,
        save_best_only=True,
    )
    es = EarlyStopping(monitor="val_loss", mode="min", verbose=0, patience=20)
    return [es, mc]

for var in y_train.features.values:
    yt = y_train.sel(features=var)
    yv = y_val.sel(features=var)
    ydims = dict(zip(yt.dims, yt.shape))
    if 'features' not in ydims:
        ydims['features'] = 1
    
#     for model in [make_model_1(var, ydims), make_model_2(var, ydims), make_model_3(var, ydims), make_model_4(var, ydims)]:
    for model in [make_model_1(var, ydims)]:
        model.summary()
        history[model.name] = model.fit(
            x_train.values,
            yt.values,
            validation_data=(x_val.values, yv.values,),
            batch_size=batch_size,
            epochs=500,
            shuffle=True,
            callbacks=make_callbacks(model.name),
        )
        

In [ ]:
h.history.keys()

In [ ]:
scores = [
    #     "val_loss",
    "val_rmse",
    "val_mse",
    "val_r_square",
    "val_bias",
]

fig, axes = plt.subplots(ncols=len(scores), nrows=4, figsize=(20, 16))
for i, var in enumerate(y_train.features.values):
    for j, score in enumerate(scores):
        # plot training history
        plt.sca(axes[i, j])
        for model, h in history.items():
            if var in model:
                plt.plot(h.history[score], label=model)
        plt.ylabel(var)
        # plt.yscale('log')
        # plt.xscale('log')
        plt.legend()
plt.tight_layout()

In [ ]:
print("here")

In [ ]:
from joblib import dump

hdump = {}

for k, v in history.items():
    hdump[k] = v.history

In [ ]:
dump(hdump, "train_history.joblib")

In [ ]:
from joblib import load

In [ ]:
hdump = load("train_history.joblib")

In [ ]:
scores_df = pd.DataFrame()
for var in y_train.features.values:
    var_scores = {}
    for model, scores in hdump.items():
        if var in model:
            key = model.replace("_" + var, "")
            var_scores[key] = max(scores["r_square"])
    scores_df[var] = pd.Series(var_scores)

In [ ]:
scores_df.T.plot.bar()
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))

In [ ]:
scores_df.plot.bar()
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))

Based on this analysis, it seems like the 3_layer_lstm_wide model is performing best for all four variables. We'll go with that for now.